# Cleaning

In [1]:
import os
import pandas as pd
from tqdm import tqdm

import pathlib
from selectolax.parser import HTMLParser
from urllib.parse import parse_qs

Semua data yang berhasil di-*scrape* disimpan di folder `./raw`

In [2]:
len(os.listdir('raw'))

5052

Buat dataframe dan fungsi untuk menghasilkan+menambahkan data bersih dari setiap file ke dataframe

In [3]:
def get_href_param(a_tag):
    return parse_qs(a_tag.attributes.get('href').split('?')[1])

def formatted(filename):
    html = pathlib.Path(filename).read_text()
    tag_a = HTMLParser(html).css('a')

    return [
        {k:v[0] for (k,v) in get_href_param(a).items()}
        for a in tag_a
    ]


In [4]:
all_records = []
for fname in tqdm(sorted(os.listdir('raw'))):
    if fname[-4:]!='html':
        continue
    all_records.extend(formatted(f'raw/{fname}'))

100%|██████████| 5052/5052 [00:02<00:00, 2122.99it/s]


In [5]:
katalog = pd.DataFrame(all_records)

In [6]:
katalog

,tgl,ot,lat,lon,ketlat,ketlon,depth,mag,remark,status,strike1,dip1,rake1,strike2,dip2,rake2
0,2008/11/01,21:02:43.058,9.18,119.06,S,E,10,4.9,"Sumba Region, Indonesia",unset,NaN,NaN,NaN,NaN,NaN,NaN
1,2008/11/01,20:58:50.248,6.55,129.64,S,E,10,4.6,Banda Sea,unset,NaN,NaN,NaN,NaN,NaN,NaN
2,2008/11/01,17:43:12.941,7.01,106.63,S,E,121,3.7,"Java, Indonesia",unset,NaN,NaN,NaN,NaN,NaN,NaN
3,2008/11/01,16:24:14.755,3.30,127.85,S,E,10,3.2,"Seram, Indonesia",unset,NaN,NaN,NaN,NaN,NaN,NaN
4,2008/11/01,16:20:37.327,6.41,129.54,S,E,70,4.3,Banda Sea,unset,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87710,2022/08/31,12:19:13.873,8.32,116.26,S,E,10,2.2,"Sumbawa Region, Indonesia",unset,NaN,NaN,NaN,NaN,NaN,NaN
87711,2022/08/31,11:39:49.556,9.52,124.56,S,E,10,3.1,Timor Region,unset,NaN,NaN,NaN,NaN,NaN,NaN
87712,2022/08/31,06:48:34.229,2.32,127.03,N,E,21,3.0,Northern Molucca Sea,unset,NaN,NaN,NaN,NaN,NaN,NaN
87713,2022/08/31,03:39:31.487,2.93,98.55,N,E,10,2.6,"Northern Sumatra, Indonesia",unset,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
for column in katalog:
    # mencari kolom dengan kurang dari 30 data unik
    z = set(katalog[column])
    if len(z)>30: continue

    print(column)
    print(z)

# ternyata bisa dihapus....

# katalog['ketlon'] sebenarnya hanya berisi 'E' dan 'W'
# Namun karena geografi Indonesia hanya berisi 'E'
# dan data selain 'E' hanya berisi data kotor... 
katalog = katalog[katalog['ketlon']=='E'] 

# bersihkan formatting data pada kolom remark
clean = lambda v: v\
    .replace('+', ' ')\
    .replace('%2C', '-')\
    .replace(', ', ' - ')\
    .strip()
katalog['remark'] = katalog['remark'].apply(clean)

# gabungkan data kolom ketlat dan kolom lat
# lat bernilai positif jika ketlat bernilai 'N'
# dan negatif jika ketlat bernilai 'S'
signed = lambda v: v[0] if v[1]=='N' else f'-{v[0]}'
# katalog['lat'] = katalog[['lat', 'ketlat']].apply(signed, axis=1)

# hapus kolom-kolom yang tidak digunakan
katalog = katalog.drop(columns=['ketlon', 'ketlat', 'status'])

katalog.to_csv('katalog_gempa.csv', index=False)

ketlat
{nan, 'for:', 'N', 'alternative', 'level', '9.5', 'the', ']', 'Lon', '[service://]location', 'send', 'print', 'given', '110', 'Warning', 'Use', 'until', 'S', 'supported', '--verbosity=4'}
ketlon
{nan, 'file', 'syslog', 'output', 'mode:', 'the', 'M', 'from', 'E', '(InaTEWS)', 'path', 'verbosity', 'component', 'than', 'ID', 'drivers', 'instead', 'to'}
status
{nan, '.....', 'unset'}


/tmp/ipykernel_6354/2309260544.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  katalog['remark'] = katalog['remark'].apply(clean)


In [8]:
katalog.fillna('')

,tgl,ot,lat,lon,depth,mag,remark,strike1,dip1,rake1,strike2,dip2,rake2
0,2008/11/01,21:02:43.058,9.18,119.06,10,4.9,Sumba Region - Indonesia,,,,,,
1,2008/11/01,20:58:50.248,6.55,129.64,10,4.6,Banda Sea,,,,,,
2,2008/11/01,17:43:12.941,7.01,106.63,121,3.7,Java - Indonesia,,,,,,
3,2008/11/01,16:24:14.755,3.30,127.85,10,3.2,Seram - Indonesia,,,,,,
4,2008/11/01,16:20:37.327,6.41,129.54,70,4.3,Banda Sea,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87710,2022/08/31,12:19:13.873,8.32,116.26,10,2.2,Sumbawa Region - Indonesia,,,,,,
87711,2022/08/31,11:39:49.556,9.52,124.56,10,3.1,Timor Region,,,,,,
87712,2022/08/31,06:48:34.229,2.32,127.03,21,3.0,Northern Molucca Sea,,,,,,
87713,2022/08/31,03:39:31.487,2.93,98.55,10,2.6,Northern Sumatra - Indonesia,,,,,,
